# Лабораторная работа №1: Первичное исследование данных

## 1. Постановка задачи (Business Understanding)

### Описание данных
Датасет содержит почасовые данные об аренде велосипедов за определенный период. Включает информацию о дате, погодных условиях (сезон, температура, влажность, ветер) и типах дней (праздничные/рабочие). Целевая переменная — общее количество арендованных велосипедов (`count`).

### Условный заказчик
Сервис велопроката (например, Velobike) или Департамент транспорта города.

### Задачи интеллектуального анализа (Data Mining)
1. **Прогнозирование спроса (Регрессия):** Предсказание количества велосипедов (`count`) на следующий час/день для оптимизации их распределения по станциям.
2. **Поиск аномалий:** Выявление дней с неестественно низким спросом (возможные поломки системы) или аномально высоким спросом (события, праздники).
3. **Профилирование поведения:** Анализ влияния погоды на разные категории пользователей (`casual` vs `registered`).

## 2. Паспорт датасета (Data Understanding)

### 2.1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка стиля графиков
sns.set(style="whitegrid")

# Загрузка данных (предполагаем, что скрипт генерации уже отработал)
try:
    df = pd.read_csv('../data/bikes_dirty.csv')
    print("Данные успешно загружены.")
except FileNotFoundError:
    print("Файл не найден! Запустите generate_data.py в папке data/")

# Преобразование datetime сразу, так как это ключевое поле
df['datetime'] = pd.to_datetime(df['datetime'])

### 2.2. Размер и структура

In [ ]:
print(f"Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов")

print("\nПервые 5 строк:")
display(df.head())

### 2.3. Типы признаков

In [ ]:
df.info()

**Вывод по типам:**
- `datetime`: временная метка (приведена к типу datetime64).
- `season`, `weather`: закодированы числами, по смыслу — категориальные (Nominal).
- `holiday`, `workingday`: бинарные (0/1).
- `temp`, `atemp`, `humidity`, `windspeed`: количественные непрерывные.
- `casual`, `registered`, `count`: количественные дискретные (целые числа).

Есть признаки с пропусками (количество Non-Null меньше общего числа строк).

## 3. Аудит качества данных

### 3.1. Пропуски (Missing Values)

In [ ]:
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_table = pd.concat([missing_data, missing_percent], axis=1, keys=['Total Missing', 'Percent (%)'])
missing_table = missing_table[missing_table['Total Missing'] > 0].sort_values('Percent (%)', ascending=False)

display(missing_table)

**Вывод:** Обнаружены пропуски в столбцах `weather`, `workingday`, `windspeed`. Доля пропусков около 5%. Это не критично, можно использовать импутацию (замену на моду или медиану).

### 3.2. Дубликаты

In [ ]:
duplicates_count = df.duplicated().sum()
print(f"Количество полных дубликатов строк: {duplicates_count}")

# Проверка дубликатов по времени (должно быть уникально)
time_duplicates = df.duplicated(subset=['datetime']).sum()
print(f"Дубликаты по времени (datetime): {time_duplicates}")

### 3.3. Типические проблемы значений (Describe)

In [ ]:
df.describe().T

**Наблюдения:**
1. `humidity`: Максимальное значение **250**. Это физически невозможно (максимум 100%). Явная ошибка данных.
2. `windspeed`: Минимальное значение может быть отрицательным (если попало в генерацию ошибок), что невозможно для скорости.
3. `count`: Мин = 0 (возможно), Макс - нужно проверить на выбросы.

### 3.4. Выбросы (Outliers)

In [ ]:
# Анализ выбросов для 'count' методом IQR
Q1 = df['count'].quantile(0.25)
Q3 = df['count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['count'] < lower_bound) | (df['count'] > upper_bound)]
print(f"Границы выбросов для 'count': [{lower_bound:.2f}, {upper_bound:.2f}]")
print(f"Количество выбросов: {len(outliers)}")

# Визуализация выбросов
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.boxplot(y=df['count'])
plt.title('Boxplot: Count')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['humidity'])
plt.title('Boxplot: Humidity (видна аномалия)')
plt.show()

## 4. Разведочный анализ (Mini-EDA)

### 4.1. Распределения

In [ ]:
# Распределение целевой переменной
plt.figure(figsize=(10, 5))
sns.histplot(df['count'], kde=True, bins=30)
plt.title('Распределение количества аренды (Count)')
plt.xlabel('Количество')
plt.show()

# Распределение категориального признака
plt.figure(figsize=(8, 5))
sns.countplot(x='season', data=df)
plt.title('Количество записей по сезонам')
plt.show()

**Комментарий:** Распределение `count` скошено вправо (много случаев малой аренды, длинный хвост высокой аренды). Сезоны представлены примерно равномерно.

### 4.2. Взаимосвязи

In [ ]:
# Зависимость Аренды от Температуры
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temp', y='count', data=df, hue='season', palette='viridis')
plt.title('Зависимость количества аренды от температуры')
plt.xlabel('Температура')
plt.ylabel('Количество аренд')
plt.show()

**Комментарий:** Визуально можно оценить, растет ли спрос с повышением температуры. Также видно разделение по сезонам.